# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [73]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

from config import GoogleMaps_API_key
gmaps.configure(api_key=GoogleMaps_API_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [74]:
cities = pd.read_csv("06-Python-APIs_Homework_Instructions_output_data_cities (3).csv", encoding="utf-8")
cities = cities.set_index('City')
cities.head()

,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City,,,,,,,,,
ostrovnoy,0,2,RU,1558378754,72,68.05,39.51,37.50,7.16
mahebourg,1,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
qaanaaq,2,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
zhuhai,3,0,CN,1558378755,39,40.71,112.04,44.60,4.47
cape town,4,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [75]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

In [76]:
#Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, 
                                 max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)

In [88]:
#Display Heatmap
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [89]:
narrow_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Wind Speed"] < 10) & 
                            (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]
narrow_city_df = narrow_city_df.dropna(how='any')
narrow_city_df.reset_index(inplace=True)
narrow_city_df.head()    

,City,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ponta do sol,70,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
1,cidreira,88,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
2,morondava,138,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
3,vaini,139,0,IN,1558378780,88,15.34,74.49,78.90,1.59
4,nador,176,0,MA,1558378499,40,35.17,-2.93,75.20,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [84]:
hotel_df = []

for i in range(len(narrow_city_df)):
    lat = narrow_city_df.loc[i]['Lat']
    lng = narrow_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": GoogleMaps_API_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_df.append(jsn['results'][0]['name'])
    except:
        hotel_df.append("")
narrow_city_df["Hotel Name"] = hotel_df
narrow_city_df = narrowed_city_df.dropna(how='any')
narrow_city_df.head()

,City,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,ponta do sol,70,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Capitólio
1,cidreira,88,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,CIDREIRA
2,morondava,138,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Morondava
3,vaini,139,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Jagalbet
4,nador,176,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Nador


In [85]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrow_city_df.iterrows()]
locations = narrow_city_df[["Lat", "Lng"]]

In [90]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

In [91]:
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))